In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import table
import datetime as dt 
# dt as time_series operation
%matplotlib inline
# for display dataframe
from IPython.display import display
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)
# ignore warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
# storage
from google.datalab import Context
import google.datalab.bigquery as bq
try:
  from StringIO import StringIO
except ImportError:
  from io import BytesIO as StringIO
# ML
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import xgboost as xgb
import pickle

In [30]:
!pip install -U xgboost

Requirement already up-to-date: xgboost in /usr/local/envs/py3env/lib/python3.5/site-packages (0.81)


In [3]:
xgb.__version__

'0.81'

# Model Predction

In [15]:
X_val = pd.read_csv('X_val.csv')
Y_val = pd.read_csv('Y_val.csv')
X_train = pd.read_csv('X_train.csv')
Y_train = pd.read_csv('Y_train.csv')

In [18]:
# model
xgb_watchlist = [(X_train, Y_train), (X_val, Y_val)]
model = xgb.XGBClassifier(learning_rate=0.08, max_depth=4,n_estimators=300,\
                 subsample=0.5, seed=2,missing=-1)
model.fit(X_train, Y_train,eval_set=xgb_watchlist,eval_metric='logloss',
            early_stopping_rounds=20,verbose=70)
# caculating E_val

model_probs = model.predict_proba(X_val)
    # [:,1] to show the prob to is_churn = 1
model_val_score = log_loss(Y_val,model_probs[:,1])
print(model_val_score)

[0]	validation_0-logloss:0.627331	validation_1-logloss:0.627325
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 20 rounds.
[70]	validation_0-logloss:0.133999	validation_1-logloss:0.132919
[140]	validation_0-logloss:0.132996	validation_1-logloss:0.132251
[210]	validation_0-logloss:0.132521	validation_1-logloss:0.132141
Stopping. Best iteration:
[194]	validation_0-logloss:0.132622	validation_1-logloss:0.132135

0.13213545218176795


In [28]:
X_val['model_pred'] = model_probs[:,1]
X_val['is_churn'] = Y_val['is_churn']
X_val[X_val['is_churn'] == 1.0]

,six_month_day_listen,six_month_satis,registered_via,age_under_26,last_last_churn,client_level_code,last_auto_renew,one_month_day_listen,model_pred,is_churn
39,165.0,0.824732,9.0,0,0.0,3,0,19.0,0.119513,1.0
61,42.0,0.913544,9.0,1,0.0,3,0,17.0,0.306601,1.0
79,1.0,0.357143,9.0,-1,-1.0,0,0,-1.0,0.840936,1.0
139,170.0,0.914198,9.0,0,-1.0,0,0,26.0,0.210925,1.0
140,121.0,0.745259,4.0,1,0.0,3,1,16.0,0.038729,1.0
152,34.0,0.564879,3.0,1,1.0,1,0,17.0,0.491203,1.0
199,171.0,0.679492,9.0,1,-1.0,0,0,26.0,0.248594,1.0
203,89.0,0.881744,3.0,0,0.0,1,0,14.0,0.339634,1.0
270,108.0,0.826468,9.0,0,0.0,4,1,3.0,0.036533,1.0
286,-1.0,-1.000000,-1.0,-1,0.0,4,1,-1.0,0.016832,1.0


# Model Prediction (Lift)

## Model Explainability (Global)
* Permutation importance

## Model Explainability (Local) 
* Tree surrogate